What is an object?

A container:
- contains data or state, accessed through attributes
- contains functionality or behavior, accessed through methods

What is a class?

Classes are themselves objects:
- The have attributes (class name)
- And behavior (how to create the class)

If a class is an object, but objects are created from classes, how are classes created?

They are created from the `type` metaclass.

### Class Attributes

__Defining/Retrieving/Setting Attributes of Classes__

In [2]:
class MyClass:
    language = 'python'
    version = '3.6'

In [4]:
getattr(MyClass, 'version', '3.0')

'3.6'

In [5]:
MyClass.version

'3.6'

In [6]:
setattr(MyClass, 'version', '3.8')

In [7]:
MyClass.version = '3.8'

In [8]:
MyClass.x = 100

Where is this state stored?

In a dictionary! Referred to as the class namespace

In [9]:
MyClass.__dict__

mappingproxy({'__module__': '__main__',
              'language': 'python',
              'version': '3.8',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None,
              'x': 100})

In [10]:
# You can also delete attributes (most of the time)
delattr(MyClass, 'x')

In [11]:
del MyClass.language

### Callable Class Attributes

Since attrbutes can be any object, that means they can also be callable.

In [20]:
class MyClass:
    version = "3.6"
    
    def hello():
        print('Hello')

In [21]:
MyClass.__dict__['hello']

<function __main__.MyClass.hello()>

In [22]:
MyClass.hello()

Hello


### Classes are Callables

When we define a class, Python adds behaviors to make that class callable. The return of this call will be an object of the type of the class. (Instantiating the class)

In [23]:
my_obj = MyClass()
isinstance(my_obj, MyClass)

True

The class object has its own namespace that is distinct from the namespace of the class itself.

In [24]:
my_obj.__dict__

{}

In [25]:
my_obj.__class__

__main__.MyClass

### Data Attributes

If Python doesn't find an attribute in an instance of a class, it will look for that attribute in the class namespace.

In [28]:
# The instance dict is empty
print(my_obj.__dict__)

# But the class dict has values we want
print(MyClass.__dict__)

{}
{'__module__': '__main__', 'version': '3.6', 'hello': <function MyClass.hello at 0x7f0985091ca0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


In [27]:
# If we try and access the version attribute, Python will get it from the class object instead.
my_obj.version

'3.6'

### Function Attributes

We can see attributes that are functions are treated differently

In [29]:
MyClass.hello

<function __main__.MyClass.hello()>

In [31]:
# Here we see the hello function for the instance, is referred to as a 'bound method'
my_obj.hello

<bound method MyClass.hello of <__main__.MyClass object at 0x7f09851f9100>>

`method` is an actual object type in Python, it is callable, and bound to some object. That object is passed to the method as its first parameter.

In [34]:
# If we call hello() on the instance, the instance object will be injected as 
# the first parameter, but the function does not expect any parameters so it will error
my_obj.hello()

TypeError: hello() takes 0 positional arguments but 1 was given

Methods are objects that combine an instance of some class, and a function.

Method objects have attributes:
- `__self__`: the instacne the method is bound to
- `__func__`: the original function defined in the class

In [35]:
# When we define an instacne method, we must account for the first argument
# which will be the instance object itself
class Person:
    def hello(self):
        print("Hello")
        
p = Person()

In [39]:
print(hex(id(p)))
p.hello.__self__

0x7f0985422dc0


Through this first argument which is the instance object, we can access the instance namespace.

In [42]:
class MyClass:
    language = 'python'
    version = '3.0'
    
    def get_version(self):
        print(f"Language: {MyClass.language}, Version: {self.version}")

In [46]:
new_version = MyClass()
new_version.version = "3.7"
new_version.get_version()

Language: python, Version: 3.7


### Initializing Class Instances

When we instantiate a class, Python does two things:
- creates a new instance
- initializes the namespace

We can provide a custom initializer that Python will use.

In [1]:
class MyClass:
    language = 'python'
    
    def __init__(self, version):
        self.version = version

By the time `__init__` is called, Python has already created the instance object and a namespace for it. Then `__init__` is called as a bound method to the new instance. 

We can optionally specify a custom function for creating the instance object (before `__init__` is called): `__new__`

### Creating Attributes at Run-Time

If we set an function attribute on an already created object, that function will not be a bound method.

In [2]:
my_obj = MyClass("3.0")
my_obj.hello = lambda: print("Hello")

my_obj.hello

<function __main__.<lambda>()>

But you can create a bound method at run-time by using a `MethodType`.

In [3]:
from types import MethodType

In [4]:
my_obj.get_version = MethodType(lambda self: f"Version: {self.version}", my_obj)

In [5]:
my_obj.get_version()

'Version: 3.0'

### Properties

We saw that we can create "bare" attributes in classes and instances.

```
class MyClass:
    def __init__(self, version):
        self.version = version
```

However, in most languages, direct access to attributes is discouraged. Instead, the convention is to make an attribute "private" and create public getter/setter methods. Note that "private" doesn't really exist in Python, so we use the naming convention of prefixing an underscore for attributes intended to be private.

In [8]:
class MyClass:
    def __init__(self, version):
        self._version = version
        
    def get_version(self):
        return self._version
    
    def set_version(self, value):
        self._version = value

We can use the `property` class to improve this approach by not having to call the get/set methods explictly, but instead interact with the attribute as if it was a "bare attribute"

In [13]:
class MyClass:
    def __init__(self, version):
        self._version = version
        
    def get_version(self):
        print("Getting version")
        return self._version
    
    def set_version(self, value):
        print("Setting version")
        self._version = value
        
    version = property(fget=get_version, fset=set_version)

In [14]:
my_obj = MyClass("3.0")
my_obj.version = "3.9"
my_obj.version

Setting version
Getting version


'3.9'

__The `property` Class__

`property` is a class and its constructor has a few parameters:
- `fget`: specifies function to get instance properties
- `fset`: specifies function to set instance properties
- `fdel`: specifies function to call when deleting instance properties
- `doc`: string representing the docstring for a property

In general, we start with bare attributes, and implement them as properties later if needed. By using the `property` class, we can change access to properties without changing the interface.

### Property Decorators

The `property` class defines methods (`getter, setter, deleter`) that can take a callable as an argument and returns the instance with the appropriate method now set.

In [16]:
class MyClass:
    def __init__(self, version):
        self._version = version
    
    @property
    def version(self):
        return self._version
    
    @version.setter
    def version(self, value):
        self._version = value

### Read-Only and Computed Properties

To create a read-only property, we just need to create a property with only the get accessor defined.

In [17]:
from math import pi

class Circle:
    def __init__(self, r):
        self.r = r
        
    @property
    def area(self):
        return pi * (self.r ** 2)

In [18]:
c = Circle(10)
c.area

314.1592653589793

In [19]:
# Caching result of area for a given radius
from math import pi

class Circle:
    def __init__(self, r):
        self._r = r
        self._area = None
        
    @property
    def radius(self):
        return self._r
    
    @radius.setter
    def radius(self, r):
        if r < 0:
            raise ValueError("Radius cannot be less than 0")
        else:
            self._r = r
            self._area = None
    
    @property
    def area(self):
        if self._area is None:
            self._area = pi * (self._r ** 2)
        else:
            return self._area

### Deleting Properties

In [20]:
class Circle:
    def __init__(self, color):
        self._color = color
        
    @property
    def color(self):
        return self._color
    
    @color.deleter
    def color(self):
        # Removes from the instance object, not the class
        del self._color

### Class and Static Methods

__Class Methods__

In [22]:
class MyClass:
    def hello():
        print("hello")
        
    def instance_hello(self):
        print(f"hello from {self}")
    
    @classmethod
    def class_hello(cls):
        print(f"hello from {cls}")

In [23]:
my_obj = MyClass()

In [28]:
my_obj.instance_hello

<bound method MyClass.instance_hello of <__main__.MyClass object at 0x7f25df2cefd0>>

In [27]:
my_obj.class_hello

<bound method MyClass.class_hello of <class '__main__.MyClass'>>

__Static Methods__

Can we define a function in a class that will never be bound to either an instance or the class?

Note that static methods are not commonly used and in some circles their use is discouraged entirely.

In [32]:
class Circle:
    @staticmethod
    def help():
        return 'heres some help'

In [33]:
c = Circle()
c.help

<function __main__.Circle.help()>

### Built-in and Standard Types

__The `types` Module__

Some standard types are not available directly in the built-ins.

In [34]:
import types

def my_func(): ...
    
isinstance(my_func, types.FunctionType)

True

In [35]:
types.ModuleType
types.GeneratorType

generator

### Class Body Scope

Inside the class body scope, the *labels* for functions will reside within the scope, but the function objects those labels point to will reside in the classes containing scope (module scope)

In [36]:
# module scope: Python, p

class Python: # class body scope: kingdom, phylum, __init__, speak
    kingdom = 'animalia'
    phylum = 'chordata'
    
    def __init__(self, species):
        self.species = species
        
    def speak(self):
        return 'sss'
    
p = Python('monty')

In [38]:
# This example kind of shows where the scopes of the function objects reside 
def callable_1(self, species): ...
def callable_2(self): ...
    
class Python:
    __init__ = callable_1
    speak = callable_2